# Developed by
Стефан Трајанов 221175

Филип Трајановиќ 221121

Глигорчо Трајков 221274



In [ ]:
# First we install the required packages
!pip install selenium
!pip install beautifulsoup4
!pip install pandas xlrd

Importing selenium

In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup

browser = webdriver.Chrome()

We open a random stock history page and accept the cookies to start our scraping.

In [101]:
browser.get('https://www.mse.mk/mk/stats/symbolhistory/kmb')
browser.execute_script("location.href='/mk/Home/AcceptCookiesConsent'");
browser.get("https://www.mse.mk/mk/stats/symbolhistory/kmb")

Finding all the stock codes.

In [102]:
stock_codes = browser.find_element(By.CSS_SELECTOR, "#Code").text.split("\n")
stock_codes = [string for string in stock_codes if not any(char.isdigit() for char in string)]
print(stock_codes)


['ADIN', 'ALK', 'ALKB', 'AMEH', 'APTK', 'ATPP', 'AUMK', 'BANA', 'BGOR', 'BIKF', 'BIM', 'BLTU', 'CBNG', 'CDHV', 'CEVI', 'CKB', 'CKBKO', 'DEBA', 'DIMI', 'EDST', 'ELMA', 'ELNC', 'ENER', 'ENSA', 'EUHA', 'EUMK', 'EVRO', 'FAKM', 'FERS', 'FKTL', 'FROT', 'FUBT', 'GALE', 'GDKM', 'GECK', 'GECT', 'GIMS', 'GRDN', 'GRNT', 'GRSN', 'GRZD', 'GTC', 'GTRG', 'IJUG', 'INB', 'INHO', 'INOV', 'INPR', 'INTP', 'JAKO', 'JUSK', 'KARO', 'KDFO', 'KJUBI', 'KKST', 'KLST', 'KMB', 'KMPR', 'KOMU', 'KONF', 'KONZ', 'KORZ', 'KPSS', 'KULT', 'KVAS', 'LAJO', 'LHND', 'LOTO', 'LOZP', 'MAGP', 'MAKP', 'MAKS', 'MB', 'MERM', 'MKSD', 'MLKR', 'MODA', 'MPOL', 'MPT', 'MPTE', 'MTUR', 'MZHE', 'MZPU', 'NEME', 'NOSK', 'OBPP', 'OILK', 'OKTA', 'OMOS', 'OPFO', 'OPTK', 'ORAN', 'OSPO', 'OTEK', 'PELK', 'PGGV', 'PKB', 'POPK', 'PPIV', 'PROD', 'PROT', 'PTRS', 'RADE', 'REPL', 'RIMI', 'RINS', 'RZEK', 'RZIT', 'RZIZ', 'RZLE', 'RZLV', 'RZTK', 'RZUG', 'RZUS', 'SBT', 'SDOM', 'SIL', 'SKON', 'SKP', 'SLAV', 'SNBT', 'SNBTO', 'SOLN', 'SPAZ', 'SPAZP', 'SPOL', 

Gathering all the stock data with beautiful soap

In [103]:
import csv
all_company_data = []

headers = ['КОМПАНИЈА', 'Датум', 'Цена на последна трансакција', 'Мак', 'Мин', 'Просечна цена', '%пром.', 'Количина', 'Промет во БЕСТ во денари', 'Вкупен промет во денари']
file_path = 'output_file.csv'


with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    for stock_code in stock_codes:
        browser.get(f'https://www.mse.mk/mk/stats/symbolhistory/{stock_code}')
        for i in range(1,12):
            try:
                year = 2025 - i
                new_start_date = "01.1." + str(year)
                new_end_date = "31.12." + str(year)
                
                start_date = browser.find_element(By.CSS_SELECTOR, "#FromDate")
                end_date = browser.find_element(By.CSS_SELECTOR, "#ToDate")
                search_button = browser.find_element(By.CSS_SELECTOR, "#report-filter-container ul .container-end input")
                
                browser.execute_script("arguments[0].setAttribute('value', arguments[1])", start_date, new_start_date)
                browser.execute_script("arguments[0].setAttribute('value', arguments[1])", end_date, new_end_date)
                search_button.click()
                
                table_of_data = browser.find_element(By.CSS_SELECTOR, "#resultsTable tbody")
                soup = BeautifulSoup(table_of_data.get_attribute("innerHTML"), 'html.parser')
                rows = soup.find_all("tr")
                
                for row in rows:
                    data = row.text.split('\n')
                    data.pop() 
                    data.remove('')
                    if data.__contains__(''):
                        continue
                    data.insert(0, stock_code)
                    writer.writerow(data)
            except:
                continue
